In [9]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [18]:
%%cython

import sys
import time

import numpy as np
cimport numpy as np

import interface as bb
cimport interface as bb

import cPickle

import theano
import theano.tensor as T
import lasagne

from libc.math cimport fabs, tanh
from scipy.linalg.cython_blas cimport sgemm

cimport cython

cdef:
    int NUM_HIDDEN = 16
    float b1[16]
    float b2[4]
    float min_state[36]
    float dif_state[36]

cdef float alpha = 1.0, beta = 0.0
cdef float[::1,:] s, h, y, w1, w2

s = np.empty((1,36), np.float32, order="F")
w1 = np.empty((36,NUM_HIDDEN), np.float32, order="F")
h = np.empty((1,NUM_HIDDEN), np.float32, order="F")
w2 = np.empty((NUM_HIDDEN,4), np.float32, order="F")
y = np.empty((1,4), np.float32, order="F")

cdef int NUM_CACHE = 21
cdef int cache_i = 0, cache_n = 0
cdef float[::1,:] cache_s, cache_y

cache_s = np.empty((NUM_CACHE,36), np.float32, order="F")
cache_y = np.empty((NUM_CACHE,4), np.float32, order="F")

@cython.boundscheck(False)
cdef void fast_target(float *state, int use_cache = 0):
    global cache_i, cache_n
    cdef int i, c, m, n, k, lda, ldb, ldc
    
    if use_cache == 1:
        c = 0
        while c < cache_n:
            i = 0
            while i < 36:
                if cache_s[c,i] != state[i]:
                    break
                i += 1
            if i == 36:
                for i in xrange(4):
                    y[0,i] = cache_y[c,i]
                return
            c += 1
        cache_i += 1
        if cache_i == NUM_CACHE:
            cache_i = 0
        if cache_n < NUM_CACHE:
            cache_n += 1
        for i in xrange(36):
            cache_s[cache_i,i] = state[i]
            s[0,i] = (state[i] - min_state[i]) / dif_state[i]
    else:
        for i in xrange(36):
            s[0,i] = (state[i] - min_state[i]) / dif_state[i]
    
    lda = 1
    ldb = 36
    ldc = 1
    m = 1
    n = NUM_HIDDEN
    k = 36
    sgemm("N", "N", &m, &n, &k, &alpha, &s[0,0], &lda, &w1[0,0], &ldb, &beta, &h[0,0], &ldc)
    
    for i in xrange(NUM_HIDDEN):
        h[0,i] = tanh(h[0,i] + b1[i])
    
    lda = 1
    ldb = NUM_HIDDEN
    ldc = 1
    m = 1
    n = 4
    k = NUM_HIDDEN
    sgemm("N", "N", &m, &n, &k, &alpha, &h[0,0], &lda, &w2[0,0], &ldb, &beta, &y[0,0], &ldc)
    
    if use_cache == 1:
        for i in xrange(4):
            cache_y[cache_i,i] = y[0,i]
    

@cython.boundscheck(False)
cdef int fast_action(float *state, int use_cache = 0):
    cdef int i, best_act = -1
    cdef float x, best_val = -1e9
    fast_target(state, use_cache)
    for i in xrange(4):
        x = y[0,i] + b2[i]
        if x > best_val:
            best_val = x
            best_act = i
    return best_act


@cython.boundscheck(False)
cdef float fast_value(float *state):
    cdef int i
    cdef float x, best_val = -1e9
    fast_target(state, 1)
    for i in xrange(4):
        x = y[0,i] + b2[i]
        if x > best_val:
            best_val = x
    return best_val


def dump_embedding():
    cdef int i
    global min_state, dif_state
    state_min = None
    state_dif = None
    with open('state36.pkl', 'r') as file:
        embedding = cPickle.load(file)
        state_min = embedding['state_min']
        state_dif = embedding['state_dif']
        for i in xrange(36):
            min_state[i] = state_min[i]
            dif_state[i] = state_dif[i]
        print 'Embedding:', state_min.mean(), state_dif.mean()
    return state_min, state_dif


@cython.boundscheck(False)
def dump_weights(weights):
    cdef int i, j
    for i in xrange(NUM_HIDDEN):
        for j in xrange(36):
            w1[j,i] = weights[0][j,i]
        b1[i] = weights[1][i]
    for i in xrange(4):
        for j in xrange(NUM_HIDDEN):
            w2[j,i] = weights[2][j,i]
        b2[i] = weights[3][i]


def prepare_bbox(level='train', verbose=0):
    if bb.is_level_loaded():
        bb.reset_level()
    bb.load_level('../levels/'+level+'_level.data', verbose)

cdef float _rewards[4]
cdef float _mask[4]

@cython.boundscheck(False)
cdef crollout(int epoch=0, float curriculum=0.7):
    cdef:
        int i, a, action, has_next, checkpoint_id, has_change
        float r, prev_score, init_state35, next_state35, next_state35_abs, prev_state35_abs
        float *state
    
    init_state35 = bb.c_get_state()[35]
    checkpoint_id = bb.create_checkpoint()
    for a in xrange(4):
        _rewards[a] = 0
        _mask[a] = 0
        
        prev_score = bb.c_get_score()
        has_next = bb.c_do_action(a)
        state = bb.c_get_state()
        next_state35 = state[35]  
        
        if init_state35 != next_state35 or np.random.rand() < curriculum:
            r = bb.c_get_score() - prev_score
            #next_state35_abs = fabs(next_state35)
            #if next_state35_abs > 0.75 and next_state35_abs > fabs(init_state35):
            #    r -= next_state35_abs
            #prev_state35_abs = next_state35_abs
            prev_score = bb.c_get_score()
            if has_next == 1:
                for i in xrange(20):
                    if epoch > 0:
                        action = fast_action(state, 1)
                    else:
                        action = 3
                    has_next = bb.c_do_action(action)
                    r += bb.c_get_score() - prev_score
                    state = bb.c_get_state()
                    #next_state35_abs = fabs(state[35])
                    #if next_state35_abs > 0.75 and next_state35_abs > prev_state35_abs:
                    #    r -= next_state35_abs
                    #prev_state35_abs = next_state35_abs
                    prev_score = bb.c_get_score()
                    if has_next == 0:
                        break
                if has_next == 1 and epoch > 0:
                    r += fast_value(state)
            
            _rewards[a] = r
            _mask[a] = 1
        
        bb.load_from_checkpoint(checkpoint_id)
    bb.clear_all_checkpoints()


@cython.boundscheck(False)
def rollout(epoch=0, curriculum=0.7):
    cdef int i
    crollout(epoch, curriculum)
    rewards = np.empty(4, dtype=np.float32)
    mask = np.empty(4, dtype=np.float32)
    for i in xrange(4):
        rewards[i] = _rewards[i]
        mask[i] = _mask[i]
    if (rewards == 0).all():
        return None, None
    return rewards, mask

BATCH_SIZE = 64

def build_model():
    
    l_input = lasagne.layers.InputLayer((None, 36))
    l_input = lasagne.layers.GaussianNoiseLayer(l_input, sigma=0.01)
    l_hidden = lasagne.layers.DenseLayer(
        l_input,
        num_units=NUM_HIDDEN,
        W=lasagne.init.Normal(),
        nonlinearity=lasagne.nonlinearities.tanh
    )
    l_out = lasagne.layers.GaussianNoiseLayer(l_hidden, sigma=0.01)
    #l_out = lasagne.layers.DropoutLayer(l_out, p=0.5)
    l_out = lasagne.layers.DenseLayer(
        l_out,
        num_units=4,
        W=lasagne.init.Normal(),
        nonlinearity=None
    )

    state = T.matrix('state')
    target = T.matrix('target')
    mask = T.matrix('mask')

    prediction = lasagne.layers.get_output(l_out, state)
    loss = T.mean(mask * (prediction - target)**2)
    #l2_penalty = T.sum(l_hidden.W**2) + T.sum(l_out.W**2)
    #loss = loss + 0.00001 * l2_penalty
    params = lasagne.layers.helper.get_all_params(l_out)
    grads = T.grad(loss, params)
    #updates = lasagne.updates.rmsprop(grads, params, .0001, .95)
    updates = lasagne.updates.adam(grads, params, .0001)
    train = theano.function([state, target, mask], loss, updates=updates)
    grads = [T.sqrt(T.sum(g**2)) for g in grads]
    debug = theano.function([state, target, mask], grads)
    
    return l_out, train, debug


@cython.boundscheck(False)
def policy_iteration(n_epochs=20):
    cdef int epoch, action
    
    best_score = 3000
    
    X = []
    Y = []
    M = []
    
    curriculum = 0.3
    curriculum_max = 0.6
    curriculum_inc = (curriculum_max - curriculum) / 10.0
    
    state_min, state_dif = dump_embedding()
    model, train, debug = build_model()
    weights_out = []
    
    def train_epoch(X, Y, M):
        cdef float loss = 0
        cdef int i = 0, e = min(5, 2 + epoch)
        for _ in xrange(e):
            N = X.shape[0]
            I = np.random.permutation(N)
            X = X[I]
            Y = Y[I]
            M = M[I]
            N = int(N / BATCH_SIZE)
            for b in xrange(N):
                Xb = X[b*BATCH_SIZE:(b+1)*BATCH_SIZE]
                Yb = Y[b*BATCH_SIZE:(b+1)*BATCH_SIZE]
                Mb = M[b*BATCH_SIZE:(b+1)*BATCH_SIZE]
                loss += train(Xb, Yb, Mb)
                i += 1
                if np.isnan(loss):
                    raise TypeError("Loss function return NaN!")

        W = lasagne.layers.get_all_param_values(model)
        
        G = debug(Xb, Yb, Mb)
        for w, g in zip(W, G):
            print "{:10s} \t {:8.5f} \t {:8.5f}".format(w.shape, np.sqrt((w**2).sum()), float(g))

        print "Updates {}, Loss = {:.4f}".format(i, loss / i)
        
        return W
    
    for epoch in range(n_epochs):
        
        start = time.time()
        
        prepare_bbox('train')
        while True:
            
            rewards, mask = rollout(epoch, curriculum)
            if rewards is not None:
                state = (bb.get_state() - state_min) / state_dif
                X.append(state)
                Y.append(rewards)
                M.append(mask)
            
            if epoch > 0:
                action = fast_action(bb.c_get_state(), 1)
            else:
                action = np.random.randint(4)

            if bb.c_do_action(action) == 0:
                train_score = bb.finish(verbose=0)
                break
            
        Xnew = np.array(X).astype(np.float32)
        Ynew = np.array(Y).astype(np.float32)
        Mnew = np.array(M).astype(np.float32)

        del X[:]
        del Y[:]
        del M[:]

        if epoch > 0:
            Xa = np.concatenate([Xnew, Xold], axis=0)
            Ya = np.concatenate([Ynew, Yold], axis=0)
            Ma = np.concatenate([Mnew, Mold], axis=0)
        else:
            Xa = Xnew
            Ya = Ynew
            Ma = Mnew

        Xold = Xnew
        Yold = Ynew
        Mold = Mnew

        weights = train_epoch(Xa, Ya, Ma)
        weights_out.append(weights)
        
        print 'Epoch: {}, sample prob: {}, time: {}'.format(epoch, curriculum, int(time.time() - start))
        test_score, _ = test(weights)
        print
        sys.stdout.flush()
        
        dump_weights(weights)
        
        if test_score > best_score:
            with open('best_weights' + str(NUM_HIDDEN) + '.pkl', 'w') as f:
                cPickle.dump(weights, f)
            best_score = test_score
            
        curriculum = min(curriculum_max, curriculum + curriculum_inc)
    
    return weights_out

def test(weights):
    cdef:
        int action, has_next
    
    dump_weights(weights)
    results = []
    for lvl in  ('train', 'test'):
        prepare_bbox(lvl)
        has_next = 1
        while has_next:
            action = fast_action(bb.c_get_state(), 0)
            has_next = bb.c_do_action(action)
        results.append(bb.finish(verbose=0))
    print 'average  {:.2f}, test {:.2f}, train {:.2f}'.format(0.5*sum(results), results[1], results[0])
    return results

In [19]:
W = policy_iteration(50)

Embedding: -5.73218 9.42407
(36, 16)   	  0.90372 	  0.68124
(16,)      	  0.08046 	  0.19938
(16, 4)    	  0.49508 	  0.61496
(4,)       	  0.04463 	  1.05380
Updates 27440, Loss = 14.9823
Epoch: 0, c: 0.3, time: 40
average  19.51, test 312.67, train -273.65

(36, 16)   	  2.55809 	  0.44457
(16,)      	  0.25087 	  0.11353
(16, 4)    	  1.40660 	  0.35460
(4,)       	  0.08820 	  0.17449
Updates 87507, Loss = 7.5495
Epoch: 1, c: 0.33, time: 104
average  -3365.76, test -3571.39, train -3160.14

(36, 16)   	  4.18970 	  0.08154
(16,)      	  0.44802 	  0.01658
(16, 4)    	  1.91879 	  0.07149
(4,)       	  0.07196 	  0.02623
Updates 125952, Loss = 0.6672
Epoch: 2, c: 0.36, time: 156
average  -410.76, test -324.98, train -496.53

(36, 16)   	  5.63463 	  0.24911
(16,)      	  0.75491 	  0.06929
(16, 4)    	  2.68036 	  0.10960
(4,)       	  0.10697 	  0.02414
Updates 162935, Loss = 0.3198
Epoch: 3, c: 0.39, time: 191
average  951.20, test 1027.20, train 875.21

(36, 16)   	  6.64459 	  

In [20]:
import cPickle
with open('list_weights_hid16.pkl', 'wb') as f:
    cPickle.dump(np.arraW, f)

In [25]:
beest_score = 0
best_weights = None
for w in W:
    tr_score, test_score = test(w)
    if test_score > beest_score:
        beest_score = test_score
        best_weights = w
print '\nbest weights:'
test(best_weights)

average  19.51, test 312.67, train -273.65
average  -3365.76, test -3571.39, train -3160.14
average  -410.76, test -324.98, train -496.53
average  951.20, test 1027.20, train 875.21
average  856.16, test 942.58, train 769.73
average  1551.86, test 1678.06, train 1425.65
average  1771.60, test 1769.76, train 1773.43
average  1774.89, test 1944.35, train 1605.43
average  1634.33, test 1709.59, train 1559.06
average  2315.14, test 2364.78, train 2265.49
average  2718.01, test 2932.63, train 2503.38
average  2609.04, test 2691.24, train 2526.84
average  2739.11, test 2913.95, train 2564.28
average  2538.15, test 2675.91, train 2400.39
average  2471.45, test 2543.74, train 2399.15
average  2696.38, test 2812.47, train 2580.29
average  2627.30, test 2767.25, train 2487.34
average  2496.33, test 2643.31, train 2349.36
average  2691.44, test 2812.85, train 2570.03
average  1868.19, test 1935.78, train 1800.59
average  2398.02, test 2572.09, train 2223.95
average  2479.20, test 2611.98, train 2

[2611.492431640625, 2959.52294921875]

In [26]:
import cPickle
with open('best_weights_hid16.pkl', 'wb') as f:
    cPickle.dump(best_weights, f)